In [102]:
# Import dependencies
import pandas as pd
import os
from dotenv import load_dotenv    # from Karen's code
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import select      # Not used
import psycopg2
import csv   # Not used
import numpy as np   # Not used
from pprint import pprint
import json


# import gzip    # Not used

In [103]:

url_metar_gz="https://aviationweather.gov/data/cache/metars.cache.csv.gz"
url_TAF_gz="https://aviationweather.gov/data/cache/tafs.cache.csv.gz"
url_airsigmets_gz="https://aviationweather.gov/data/cache/airsigmets.cache.csv.gz"

__To retrieve METAR data__

In [104]:
# Fetch, load, and decompress the data relative to metar
metar_data_df = pd.read_csv(url_metar_gz, header=5, compression='gzip')
metar_data_df.head()

,raw_text,station_id,observation_time,latitude,longitude,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,...,maxT24hr_c,minT24hr_c,precip_in,pcp3hr_in,pcp6hr_in,pcp24hr_in,snow_in,vert_vis_ft,metar_type,elevation_m
0,PASL 282027Z AUTO 00000KT 1 1/4SM BR M14/M16 A...,PASL,2023-12-28T20:27:00Z,61.6970,-157.1670,-14.0,-16.0,0,0.0,NaN,...,NaN,NaN,0.005,NaN,NaN,NaN,NaN,NaN,SPECI,64.0
1,PAPO 282027Z AUTO 17017KT 2 1/2SM -SN BR FEW01...,PAPO,2023-12-28T20:27:00Z,68.3450,-166.8070,-15.0,-17.0,170,17.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,4.0
2,PAFE 282027Z AUTO 05006KT 8SM RA OVC028 03/02 ...,PAFE,2023-12-28T20:27:00Z,56.9610,-133.9100,3.0,2.0,50,6.0,NaN,...,NaN,NaN,0.060,NaN,NaN,NaN,NaN,NaN,SPECI,1.0
3,PAMC 282026Z AUTO 00000KT 10SM FEW015 FEW022 B...,PAMC,2023-12-28T20:26:00Z,62.9570,-155.6100,-22.2,-24.4,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,101.0
4,KSUX 282026Z 33017G25KT 10SM OVC009 00/M01 A30...,KSUX,2023-12-28T20:26:00Z,42.3917,-96.3795,0.0,-1.1,330,17.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,332.0


In [105]:
# Convert the dataframe to a JSON format
# metar_json = json.loads(json.dumps(list(metar_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(metar_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [106]:
# pprint(metar_json, sort_dicts=False)

In [107]:
# Convert some columns to INT
# cols=['wind_dir_degrees','wind_speed_kt','wind_gust_kt','cloud_base_ft_agl','cloud_base_ft_agl.2',\
#     'cloud_base_ft_agl.3','vert_vis_ft','elevation_m']
# metar_data_df[cols]=metar_data_df[cols].apply(pd.to_numeric, errors='coerce', downcast='integer', axis=1) # Does not appear to convert to INT, but to FLOAT64
# metar_data_df[cols]=metar_data_df[cols].astype(int)   # Cannot convert NaN

In [108]:
# metar_data_df.columns

In [109]:
# metar_data_df.info()

In [110]:
# convert the dataframe to a dictionary then to a JSON string
metar_string=json.dumps(list(metar_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "metar_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(metar_string.replace("NaN","null"))

# file is automatically closed after the with block

In [111]:
# To save the dataframe as a csv file for future import to database (not needed anymore)
# output_path2 = os.path.join("", "metar_data.csv")
# metar_data_df.to_csv(output_path2, index=False)

# file is automatically closed after the with block

__To refresh the metar table in the Render database__

In [112]:
# Test to refresh the metar table

load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()
cursor.execute("DELETE FROM metar;")    # Deletes all the rows but keep the table
connection.commit()
cursor.close()
connection.close()

In [113]:
# Test to populate the empty table from a json file
load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()

cursor.execute("set search_path to public") # https://dba.stackexchange.com/questions/268365/using-python-to-insert-json-into-postgresql

with open('metar_data.json') as file:
    data = file.read()

query_sql = """
INSERT INTO metar SELECT * FROM
json_populate_recordset(NULL::metar, %s);
"""

cursor.execute(query_sql, (data,))
connection.commit()

__To download a new csv file made of the joining of airport data and weather data__

In [114]:
# download the output of a query across multiple tables with all the all the active public airport with or without weather information.
# Will be used to populate the makers that do not have METAR information.
# There is only one row per airport so only one runway is listed even if the airport as more.
# More parameters can be added to complete the info on the popup window.

load_dotenv()
db_url = os.environ.get("link_render")

# query="""
#     SELECT DISTINCT ON (arpt_id)
# 	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text
#     FROM apt_rwy
#     JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
#     FULL JOIN metar ON metar.station_id = apt_base.icao_id
#     WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O';
#     """

query="""
    SELECT DISTINCT ON (arpt_id)
	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text, elev, metar.visibility_statute_mi, metar.cloud_base_ft_agl
    FROM apt_rwy
    JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
    FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
    WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O'
    """

engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )

arpt_weather_query_df = pd.DataFrame(results)

# Save the df as a CSV file. Might not be needed if we only use JSON    TO BE REMOVED?
# arpt_weather_query_df.to_csv (r'airport_weather_data.csv', index = False) # place 'r' before the path name


# convert the dataframe to a dictionary then to a JSON string
arpt_weather_string=json.dumps(list(arpt_weather_query_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "airport_weather_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(arpt_weather_string.replace("NaN","null"))

# file is automatically closed after the with block



In [115]:
# arpt_weather_query_df.head()

In [116]:
# pprint(arpt_weather_string)

__To retrieve TAF data__

In [117]:
# Fetch, load, and decompress the data relative to TAF  (working but not used for now)
# taf_data_df = pd.read_csv(url_TAF_gz, header=5,index_col=False, compression='gzip',dtype='str',low_memory=False)
# taf_data_df.head()

In [118]:
# Convert the dataframe to a JSON format  (working but not used for now)
# taf_json = json.loads(json.dumps(list(taf_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(taf_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [119]:
# # convert the dataframe to a dictionary then to a JSON string  (working but not used for now)
# taf_string=json.dumps(list(taf_data_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("Resources", "taf_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(taf_string.replace("NaN","null"))

# # file is automatically closed after the with block

__To retrieve AIRMET and SIGMET polygons data__

In [120]:
# Fetch, load, and decompress the data relative to Airmet and Sigmet
airsigmet_data_df = pd.read_csv(url_airsigmets_gz, header=5, compression='gzip', encoding='utf-8')
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lon:lat points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS01 KKCI 281830 BOSV WS 281830 SIGMET VIC...,2023-12-28T18:30:00Z,2023-12-28T22:30:00Z,-67.7404:42.8342;-67.0051:40.2844;-76.907:35.6...,23000.0,37000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
1,WSUS31 KKCI 281955 SIGE CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,-87:27.48;-84.26:27.48;-83.01:24;-78.66:24;-79...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
2,WSUS32 KKCI 281955 SIGC CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,-107:49;-107:31.79;-106.52:31.78;-106.48:31.75...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
3,WSUS33 KKCI 281955 SIGW CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,-122.77:49;-123.12:48.96;-123.02:48.6;-123.16:...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
4,WAUS45 KKCI 281445 SLCS WA 281445 AIRMET SIE...,2023-12-28T14:45:00Z,2023-12-28T21:00:00Z,-109.2793:42.0545;-105.72:40.3329;-106.1019:39...,0.0,1000.0,NaN,NaN,IFR,0,AIRMET


In [121]:
# replace with <br> in raw_text
# airsigmet_data_df['raw_text'].str.replace(r' \x07','<br>')
# airsigmet_data_df.replace(r' x07','<br>', regex=True)

# To convert the points delimiting the areas into something Leaflet-friendly
for j in range(len(airsigmet_data_df)):
    test=airsigmet_data_df.iloc[j]['lon:lat points']
    if  pd.isna(test)!=True:    # Test if the cell is not NaN
        test1=test.split(';')
        for i in range(len(test1)):
            test1[i]=test1[i].split(':')    # Creates list of coordinates
            test1[i][0],test1[i][1]=float(test1[i][1]),float(test1[i][0])   # Swap lon:lat to lat:lon
        airsigmet_data_df.at[j,'lon:lat points']=test1

    # shift cells for missing column
    test2=airsigmet_data_df.iloc[j]['airsigmet_type']
    if  pd.isna(test2)==True:    # Test if the cell is NaN
        airsigmet_data_df.at[j,'airsigmet_type']=airsigmet_data_df.at[j,'severity']
        airsigmet_data_df.at[j,'severity']=airsigmet_data_df.at[j,'hazard']
        airsigmet_data_df.at[j,'hazard']=airsigmet_data_df.at[j,'movement_speed_kt']
        airsigmet_data_df.at[j,'movement_speed_kt']=airsigmet_data_df.at[j,'movement_dir_degrees']
        airsigmet_data_df.at[j,'movement_dir_degrees']=airsigmet_data_df.at[j,'max_ft_msl']
        airsigmet_data_df.at[j,'max_ft_msl']=airsigmet_data_df.at[j,'min_ft_msl']
        airsigmet_data_df.at[j,'min_ft_msl']="NaN"

  


airsigmet_data_df.rename(columns={"lon:lat points":"lat_lon_points"}, inplace=True) # Update the column name
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lat_lon_points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS01 KKCI 281830 BOSV WS 281830 SIGMET VIC...,2023-12-28T18:30:00Z,2023-12-28T22:30:00Z,"[[42.8342, -67.7404], [40.2844, -67.0051], [35...",23000.0,37000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
1,WSUS31 KKCI 281955 SIGE CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,"[[27.48, -87.0], [27.48, -84.26], [24.0, -83.0...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
2,WSUS32 KKCI 281955 SIGC CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,"[[49.0, -107.0], [31.79, -107.0], [31.78, -106...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
3,WSUS33 KKCI 281955 SIGW CONVECTIVE SIGMET.....,2023-12-28T19:55:00Z,2023-12-28T21:55:00Z,"[[49.0, -122.77], [48.96, -123.12], [48.6, -12...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
4,WAUS45 KKCI 281445 SLCS WA 281445 AIRMET SIE...,2023-12-28T14:45:00Z,2023-12-28T21:00:00Z,"[[42.0545, -109.2793], [40.3329, -105.72], [39...",0.0,1000.0,NaN,NaN,IFR,0,AIRMET


In [122]:
# Convert the dataframe to a JSON format
# airsigmet_json = json.loads(json.dumps(list(airsigmet_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(airsigmet_json, sort_dicts=False)

In [123]:
# convert the dataframe to a dictionary then to a JSON string
airsigmet_string=json.dumps(list(airsigmet_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "airsigmet_data.json")
output_path = os.path.join("", "airsigmet_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(airsigmet_string.replace("NaN","null"))

# file is automatically closed after the with block